# [Direct Preference Optimization: Your Language Model is Secretly a Reward Model (DPO)](https://arxiv.org/pdf/2305.18290.pdf)

### Reference Code 
- https://huggingface.co/docs/trl/main/en/dpo_trainer
- https://github.com/huggingface/trl/blob/main/examples/scripts/dpo.py

Therefore the final dataset object should contain these 3 entries if you use the default DPODataCollatorWithPadding data collator. 

The entries should be named:
- prompt
- chosen
- rejected

In [1]:
import os
import torch
# Set GPU device
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

os.environ['http_proxy']  = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
dpo_dataset_dict = {
    "prompt": [
        "hello",
        "how are you",
        "What is your name?",
        "What is your name?",
        "Which is the best programming language?",
        "Which is the best programming language?",
        "Which is the best programming language?",
    ],
    "chosen": [
        "hi nice to meet you",
        "I am fine",
        "My name is Mary",
        "My name is Mary",
        "Python",
        "Python",
        "Java",
    ],
    "rejected": [
        "leave me alone",
        "I am not fine",
        "Whats it to you?",
        "I dont have a name",
        "Javascript",
        "C++",
        "C++",
    ],
}

In [3]:
import torch
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    HfArgumentParser, 
    TrainingArguments
)

from typing import Dict, Optional
from trl import DPOTrainer,DPOConfig





# 1. load a pretrained model and tokenizer

In [4]:
model_name_or_path = "gpt2"
ignore_bias_buffers = False

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
if ignore_bias_buffers:
    # torch distributed hack
    model._ddp_params_and_buffers_to_ignore = [
        name for name, buffer in model.named_buffers() if buffer.dtype == torch.bool
    ]

model_ref = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token



In [5]:

# train_dataset = load_dataset("ProlificAI/social-reasoning-rlhf",split="train[:30%]")
# eval_dataset = load_dataset("ProlificAI/social-reasoning-rlhf",split="train[30%:]")



The DPO trainer expects a model of AutoModelForCausalLM, compared to PPO that expects AutoModelForCausalLMWithValueHead for the value function.

## 2. Load the Anthropic Helpful-Harmless dataset

In [6]:
def extract_anthropic_prompt(prompt_and_response):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term = "Human: "
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]

def get_hh(split: str, sanity_check: bool = False, silent: bool = False, cache_dir: str = None) -> Dataset:
    """Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts should be structured as follows:
      \n\nHuman: <prompt>\n\nAssistant:
    Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
    """
    
    dataset = load_dataset("XueyingJia/hh-rlhf-train-helpful-subset", split=split, cache_dir=cache_dir)
    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 1000)))

    def split_prompt_and_responses(sample) -> Dict[str, str]:
        prompt = extract_anthropic_prompt(sample["prompt"])
        return {
            "prompt": prompt,
            "chosen": sample["chosen"][len(prompt) :],
            "rejected": sample["rejected"][len(prompt) :],
        }

    return dataset.map(split_prompt_and_responses)

In [7]:
sanity_check = True
train_dataset = get_hh("train[30%:]", sanity_check=sanity_check)
eval_dataset = get_hh("train[:30%]", sanity_check=sanity_check)

In [8]:
train_dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

In [9]:
train_dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

In [10]:
eval_dataset


Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 1000
})

# 3. initialize training arguments:

In [11]:
learning_rate = 1e-3
per_device_train_batch_size = 8
gradient_accumulation_steps = 1
max_length= 512 
max_prompt_length = 128 
max_target_length =128 
label_pad_token_id = 100
max_steps = 400
# instrumentation
sanity_check = True
report_to = None
gradient_checkpointing = None
beta = 0.1

In [12]:
training_args = DPOConfig(
    per_device_train_batch_size=per_device_train_batch_size,
    max_steps=max_steps,
    remove_unused_columns=False,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    evaluation_strategy="steps",
    logging_first_step=True,
    logging_steps=5,  # match results in blog post
    eval_steps=500,
    output_dir="./test_learnrate_1e-1_beta_0.1_grdaient_1",
    optim="rmsprop",
    warmup_steps=150,
    report_to=report_to,
    bf16=True,
    gradient_checkpointing=gradient_checkpointing,
    # TODO: uncomment that on the next transformers release
    # gradient_checkpointing_kwargs=gradient_checkpointing_kwargs,
)

/home/jupyter-st124903/.local/lib/python3.12/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# 4. initialize the DPO trainer

In [13]:
dpo_trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=beta,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    max_length=max_length,
    max_target_length=max_target_length,
    max_prompt_length=max_prompt_length,
    generate_during_eval=True
)

/home/jupyter-st124903/.local/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_length, max_target_length, max_prompt_length, generate_during_eval. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/jupyter-st124903/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:397: UserWarning: You passed `generate_during_eval` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/home/jupyter-st124903/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:469: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/home/jupyter-st124903/.local/lib/python3.12/site-packages/trl/trainer/dpo_trainer.py:475: UserWarning: You

# 5. Train

In [14]:
dpo_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: woramethr (woramethr-asian-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss


TrainOutput(global_step=400, training_loss=1.6901643353700637, metrics={'train_runtime': 173.6204, 'train_samples_per_second': 18.431, 'train_steps_per_second': 2.304, 'total_flos': 0.0, 'train_loss': 1.6901643353700637, 'epoch': 3.2})

In [15]:
dpo_trainer.save_model("dpo_model_rlhf")

In [16]:
from huggingface_hub import HfApi
from huggingface_hub import login

In [ ]:
login(token="---")

In [18]:
model.push_to_hub("dpo_model_rlhf")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mrwhitec/dpo_model_rlhf/commit/a3fe87d10fe63a29452e70df70b8561fe7aab628', commit_message='Upload model', commit_description='', oid='a3fe87d10fe63a29452e70df70b8561fe7aab628', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mrwhitec/dpo_model_rlhf', endpoint='https://huggingface.co', repo_type='model', repo_id='mrwhitec/dpo_model_rlhf'), pr_revision=None, pr_num=None)

In [31]:
model = AutoModelForCausalLM.from_pretrained("saved_model")
tokenizer = AutoTokenizer.from_pretrained("saved_model")

In [32]:
inputs = tokenizer("What are signs of being spiritually ascended? Seems many religions, aspire to this.", return_tensors="pt")
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [33]:
generated_text

'What are signs of being spiritually ascended? Seems many religions, aspire to this.\n\nThe first sign of being spiritually ascended is the presence of God. The second sign is the presence of God. The third sign is the presence of God. The fourth sign is the presence of God. The fifth sign is the presence of God'